# CoursewareHubの登録を学認へ申請する--直接学認フェデレーションを利用する

---
CoursewareHubをSPとして学認へ申請する

## はじめに

CoursewareHubの構成要素を以下に示します。

![構成](images/cw-321-01.png)

このNotebookでは`CoursewareHub` と学認との連携に必要となる申請や、登録などについて記します。

> このNotebookでは IdP-proxyを利用せずに、直接CoursewareHubを学認のSPとして登録する手順について記しています。

## 学認にSP設置の申請を行う

申請を行う前に学認（GakuNin）の「[参加情報](https://www.gakunin.jp/join)」にてフェデレーション参加の流れを確認してください。参加するフェデレーションに従い「[テストフェデレーション参加手続き](https://www.gakunin.jp/join/test)」または「[運用フェデレーション参加手続き](https://www.gakunin.jp/join/production)」にある「学認申請システム」から「新規SP申請」を行います。

「新規SP申請」を選択すると以下のような画面が表示されます。
> キャプチャー画面はテストフェデレーションのものです。

![新規SP申請](images/cw-531-01.png)

CoursewareHubでは[SimpleSAMLphp](https://simplesamlphp.org/)を利用しています。そのため学認申請システムのテンプレートではなく、構築したCoursewareHub のメタデータを「テンプレート外メタデータ」からアップロードして申請してください。

![メタデータアップロード](images/cw-531-02.png)

申請システムにメタデータをアップロードするために、構築したCoursewareHubからメタデータのダウンロードを行います。次のセルで対象となるCoursewareHubのホスト名(FQDN)を指定してください。

In [ ]:
# (例)
# master_fqdn = 'hub.example.org'

master_fqdn = 

CoursewareHubからメタデータをダウンロードします。次のセルを実行することで表示されるリンク先にアクセスするとメタデータがダウンロードされます。

In [ ]:
print(f'https://{master_fqdn}/simplesaml/module.php/saml/sp/metadata.php/default-sp')

ダウンロードしたメタデータを学認申請システムにアップロードすると「SPメタデータ情報」の入力欄のうちメタデータに対応するものが設定されます。他の欄を入力した後に申請してください。ただし CoursewareHubでは `mail` 属性を利用するので「受信する属性情報」に `mail` を**必須** な項目として追加してください。

## 学認mAPとの連携

CoursewareHubでは利用者をグループ管理するために[学認mAP](https://meatwiki.nii.ac.jp/confluence/display/gakuninmappublic/Home)を利用します。ここでは CoursewareHub を学認mAPと連携する手順について記します。

### 申請

学認mAPの「[問い合わせ](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=8716731)」などに記されている窓口を通して、構築したSP(CoursewareHub)と学認mAPとを連携するための依頼を行ってください。

### メタデータの登録（テストフェデレーションの場合）

テストフェデレーションの場合、mAP(SP検証環境)のメタデータをCoursewareHubに登録する必要があります。

mAP利用申請を行うと、学認クラウドゲートウェイサービスサポートからmAP(SP検証環境)のメタデータが送られてきます。送られてきたメタデータを CoursewareHub に登録します。メタデータファイルをこのNotebook環境に配置し、そのパスを次のセルで指定してください。

In [ ]:
# (例)
# cgidp_metadata = './sptestcgidp-metadata.xml'

cgidp_metadata = 

操作対象となるCoursewareHubのAnsibleグループ名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にしてAnsibleグループ名を指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

SP検証環境のメタデータをCoursewareHubの環境に配置します。

In [ ]:
!ansible {ugroup_name}_manager -b -m copy -a \
    'src={cgidp_metadata} dest={{{{base_dir}}}}/metadata/cgidp-metadata.xml \
    backup=yes owner=33 group=33'

SimpleSAMLphpの設定ページでSP検証環境のメタデータが登録されたことを確認します。次のセルを実行すると表示されるリンク先をアクセスしてください。

In [ ]:
print(f'https://{master_fqdn}/simplesaml/module.php/core/frontpage_federation.php')

管理者でログインを行い「SAML 2.0 IdPメタデータ」に「GakuNin Cloud Gateway Service (SP Test) (IdP)」が追加されていることを確認してください。

> SimpleSAMLphpの管理者パスワードは「321-CoursewareHubのセットアップ.ipynb」の「6 SimpleSAMLphpの設定ファイルを更新する」にて `config.php` の `auth.adminpassword` に設定した値となります。

### SPコネクタの作成

[学認クラウドゲートウェイサービス](https://cg.gakunin.jp/)またはSP検証環境(テストフェデレーションの場合)にアクセスしてSPコネクタの作成を行います。手順の詳細については学認クラウドゲートウェイサービス利用マニュアルの「[SPコネクタを作成する](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873902)」の章を参照してください。

> テストフェデレーションSP検証環境のアドレスは利用申請後にメールなどで通知されます。

SP管理者が学認クラウドゲートウェイのウェブサイトにアクセスすると以下のような画面が表示されます。

> キャプチャー画面はテストフェデレーションのものです。

![クラウドゲートウェイ画面](images/cw-531-03.png)

「SPコネクタの作成」を選択すると入力画面が表示されるので、必要な項目を入力してください。

![新規SPコネクタ入力画面](images/cw-531-04.png)

詳細設定の「Q1 作成するSPコネクタは、検索対象にしますか？」との項目で「検索対象としない」を選択した場合は、グループ管理者がSPコネクタを見つけることが出来なくなります。グループとの接続を行うためにはSPコネクタ側からグループへ接続申請をする必要があります。

### グループをSPコネクタに接続する

「311-mAPのグループ作成.ipynb」で作成したグループをサービス(SPコネクタ)に紐づけ、グループに属するユーザがCoursewareHubサービス(SP)を利用できるようにします。

> ここでは SPコネクタの管理者とグループの管理者が同一であることを想定しています。

グループとサービス（SPコネクタ）の紐づけは、グループ側からサービスを指定する方法とサービス側（SPコネクタ）からグループを指定して接続する方法があります。ここではグループ管理者がSPコネクタ（サービス）へと接続申請を行う手順を示します。設定方法の詳細については学認クラウドゲートウェイマニュアルの「[サービスを利用する](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873789)」を参照してください。また、逆の関連付けとなるSPコネクタからグループ管理者へ接続申請を行う手順については「[SPコネクタにグループを接続する](https://meatwiki.nii.ac.jp/confluence/pages/viewpage.action?pageId=20873907)」を参照してください。

まず、グループの管理者メニューから「利用Webサービス」を選択します。

![グループ画面](images/cw-211-03.png)

「利用するWebサービスを追加」ボタンを押すと、サービスの一覧が表示されるので「3.3 SPコネクタの作成」で登録したサービスを選択してください。

> SPコネクタとして登録する際に、詳細設定の「Q1 作成するSPコネクタは、検索対象にしますか？」の値を「検索対象としない」と設定した場合はWebサービスの一覧には表示されません。

![サービス一覧画面](images/cw-211-04.png)

CoursewareHubのサービスを選択して申請を行うと承認待ちの状態となります。

![承認待ち画面](images/cw-211-05.png)

SPコネクタの設定画面から承認を行うと申請状態が「承認」となります。

![グループ画面](images/cw-211-06.png)

## CoursewareHubに学認IdPのアカウントでログインする

学認IdPのアカウントでCoursewareHubにログインできることを確認します。

次のセルを実行すると、CoursewareHubのアドレスを表示します。

In [ ]:
print(f'https://{master_fqdn}')

ログイン画面に表示されている「学認フェデレーションへ」のリンクから学認IdPを利用したログインができます。

![ログイン画面](images/cw-221-02.png)

「学認フェデレーションへ」のリンクから進むと以下のようなIdPの選択画面が表示されます。

> キャプチャー画面はテストフェデレーションのものです。

![IdP選択画面](images/cw-221-03.png)

IdPを選択しログインを行ってください。mAPグループに所属しているユーザであればJupyterNotebookが表示されます。